Creación de un proyecto básico en ambiente docker
===

* Ultima modificación: Mayo 14, 2022

Diretorio para almacenar el proyecto
---

In [1]:
#
# Crea una carpeta para el proyecto. El proyecto también puede estar alojado en
# un repositorio de GitHub.
#
!rm -rf mlruns
!rm -rf /tmp/wine_prj
!mkdir /tmp/wine_prj

Código en Python
---

In [2]:
%%writefile /tmp/wine_prj/train_elasticnet.py

#
# Puede ejecutarse en la línea de comandos como:
# $ python3 train_elasticnet.py {alpha} {l1_ratio} {verbose}
#

def load_data():

    import pandas as pd

    url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    df = pd.read_csv(url, sep=";")

    y = df["quality"]
    x = df.copy()
    x.pop("quality")

    return x, y


def make_train_test_split(x, y):

    from sklearn.model_selection import train_test_split

    (x_train, x_test, y_train, y_test) = train_test_split(
        x,
        y,
        test_size=0.25,
        random_state=123456,
    )
    return x_train, x_test, y_train, y_test


def eval_metrics(y_true, y_pred):

    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    return mse, mae, r2


def report(estimator, mse, mae, r2):

    print(estimator, ":", sep="")
    print(f"  MSE: {mse}")
    print(f"  MAE: {mae}")
    print(f"  R2: {r2}")


def run():
    #
    # Entrena un modelo sklearn ElasticNet
    #

    import sys

    from sklearn.linear_model import ElasticNet
    
    import mlflow    

    x, y = load_data()
    x_train, x_test, y_train, y_test = make_train_test_split(x, y)

    alpha = float(sys.argv[1])
    l1_ratio = float(sys.argv[2])
    verbose = int(sys.argv[3])
    
    print('Tracking directory:', mlflow.get_tracking_uri())

    with mlflow.start_run():
    
        estimator = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=12345)
        estimator.fit(x_train, y_train)
        mse, mae, r2 = eval_metrics(y_test, y_pred=estimator.predict(x_test))
        if verbose > 0:
            report(estimator, mse, mae, r2)    
            
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        
        mlflow.log_metric("mse", mse)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("r2", r2)
        
        mlflow.sklearn.log_model(estimator, "model")            


if __name__ == "__main__":
    run()

Writing /tmp/wine_prj/train_elasticnet.py


MLproject
---

In [8]:
%%writefile /tmp/wine_prj/MLproject
name: proyecto-de-demostracion

docker_env:
    image:  jdvelasq/mlflow:example

entry_points:
    main:
        parameters:
            alpha: {type: float, default: 0.1}
            l1_ratio: {type: float, default: 0.1}
            verbose: {type: integer, default: 1}
        command: 'python3 train_elasticnet.py {alpha} {l1_ratio} {verbose}'

Overwriting /tmp/wine_prj/MLproject


Docker
---

In [4]:
%%writefile /tmp/wine_prj/Dockerfile

FROM condaforge/miniforge3

RUN pip install mlflow \
    && pip install pandas \
    && pip install scikit-learn \
    && pip install cloudpickle

Writing /tmp/wine_prj/Dockerfile


Cree el contendor reemplazando su nombre de usuario:

```shell
$ docker build -t jdvelasq/mlflow:example .
$ docker push jdvelasq/mlflow:example
```



Ejecución en el ambiente local con parámetros por defecto
---

In [9]:
#
# Ejecución con parámetros por defecto
#
!mlflow run /tmp/wine_prj

2022/04/05 07:09:27 INFO mlflow.projects.docker: === Building docker image proyecto-de-demostracion ===
2022/04/05 07:09:28 INFO mlflow.projects.utils: === Created directory /var/folders/m_/gpr6p_rx0_q6rgwrbhn3zjyc0000gn/T/tmppgxebaj6 for downloading remote URIs passed to arguments of type 'path' ===
2022/04/05 07:09:28 INFO mlflow.projects.backend.local: === Running command 'docker run --rm -v /Volumes/GitHub/courses-source/notebooks/mlflow/mlruns:/mlflow/tmp/mlruns -v /Volumes/GitHub/courses-source/notebooks/mlflow/mlruns/0/88fe78cceb1649038c695bac9573f2db/artifacts:/Volumes/GitHub/courses-source/notebooks/mlflow/mlruns/0/88fe78cceb1649038c695bac9573f2db/artifacts -e MLFLOW_RUN_ID=88fe78cceb1649038c695bac9573f2db -e MLFLOW_TRACKING_URI=file:///mlflow/tmp/mlruns -e MLFLOW_EXPERIMENT_ID=0 proyecto-de-demostracion:latest python3 train_elasticnet.py 0.1 0.1 1' in run with ID '88fe78cceb1649038c695bac9573f2db' === 
Tracking directory: file:///mlflow/tmp/mlruns
ElasticNet(alpha=0.1, l1_rat

Ejecución en el ambiente local con parámetros suministrados por el usuario
---

In [10]:
!mlflow run /tmp/wine_prj -P alpha=0.2 -P l1_ratio=0.2 -P verbose=1

2022/04/05 07:09:53 INFO mlflow.projects.docker: === Building docker image proyecto-de-demostracion ===
2022/04/05 07:09:53 INFO mlflow.projects.utils: === Created directory /var/folders/m_/gpr6p_rx0_q6rgwrbhn3zjyc0000gn/T/tmpsv3vurau for downloading remote URIs passed to arguments of type 'path' ===
2022/04/05 07:09:53 INFO mlflow.projects.backend.local: === Running command 'docker run --rm -v /Volumes/GitHub/courses-source/notebooks/mlflow/mlruns:/mlflow/tmp/mlruns -v /Volumes/GitHub/courses-source/notebooks/mlflow/mlruns/0/a2421a17aba14b9cb20e7a454c4358ec/artifacts:/Volumes/GitHub/courses-source/notebooks/mlflow/mlruns/0/a2421a17aba14b9cb20e7a454c4358ec/artifacts -e MLFLOW_RUN_ID=a2421a17aba14b9cb20e7a454c4358ec -e MLFLOW_TRACKING_URI=file:///mlflow/tmp/mlruns -e MLFLOW_EXPERIMENT_ID=0 proyecto-de-demostracion:latest python3 train_elasticnet.py 0.2 0.2 1' in run with ID 'a2421a17aba14b9cb20e7a454c4358ec' === 
Tracking directory: file:///mlflow/tmp/mlruns
ElasticNet(alpha=0.2, l1_rat

In [11]:
!mlflow run /tmp/wine_prj -P alpha=0.1 -P l1_ratio=0.1 -P verbose=1

2022/04/05 07:10:00 INFO mlflow.projects.docker: === Building docker image proyecto-de-demostracion ===
2022/04/05 07:10:00 INFO mlflow.projects.utils: === Created directory /var/folders/m_/gpr6p_rx0_q6rgwrbhn3zjyc0000gn/T/tmpl9x0kyp6 for downloading remote URIs passed to arguments of type 'path' ===
2022/04/05 07:10:00 INFO mlflow.projects.backend.local: === Running command 'docker run --rm -v /Volumes/GitHub/courses-source/notebooks/mlflow/mlruns:/mlflow/tmp/mlruns -v /Volumes/GitHub/courses-source/notebooks/mlflow/mlruns/0/95dcdd265f394428af755f096a9ba42d/artifacts:/Volumes/GitHub/courses-source/notebooks/mlflow/mlruns/0/95dcdd265f394428af755f096a9ba42d/artifacts -e MLFLOW_RUN_ID=95dcdd265f394428af755f096a9ba42d -e MLFLOW_TRACKING_URI=file:///mlflow/tmp/mlruns -e MLFLOW_EXPERIMENT_ID=0 proyecto-de-demostracion:latest python3 train_elasticnet.py 0.1 0.1 1' in run with ID '95dcdd265f394428af755f096a9ba42d' === 
Tracking directory: file:///mlflow/tmp/mlruns
ElasticNet(alpha=0.1, l1_rat

In [12]:
!mlflow run /tmp/wine_prj -P alpha=0.5 -P l1_ratio=0.5 -P verbose=1

2022/04/05 07:10:06 INFO mlflow.projects.docker: === Building docker image proyecto-de-demostracion ===
2022/04/05 07:10:07 INFO mlflow.projects.utils: === Created directory /var/folders/m_/gpr6p_rx0_q6rgwrbhn3zjyc0000gn/T/tmp1kx75l1t for downloading remote URIs passed to arguments of type 'path' ===
2022/04/05 07:10:07 INFO mlflow.projects.backend.local: === Running command 'docker run --rm -v /Volumes/GitHub/courses-source/notebooks/mlflow/mlruns:/mlflow/tmp/mlruns -v /Volumes/GitHub/courses-source/notebooks/mlflow/mlruns/0/43d3554b1aff42c0b9aff136d4179446/artifacts:/Volumes/GitHub/courses-source/notebooks/mlflow/mlruns/0/43d3554b1aff42c0b9aff136d4179446/artifacts -e MLFLOW_RUN_ID=43d3554b1aff42c0b9aff136d4179446 -e MLFLOW_TRACKING_URI=file:///mlflow/tmp/mlruns -e MLFLOW_EXPERIMENT_ID=0 proyecto-de-demostracion:latest python3 train_elasticnet.py 0.5 0.5 1' in run with ID '43d3554b1aff42c0b9aff136d4179446' === 
Tracking directory: file:///mlflow/tmp/mlruns
ElasticNet(alpha=0.5, random

MLflow ui
---

Para visualizar la interfase use:

```bash
mlflow ui
```

**Nota:** En docker usar:

```bash
mlflow ui --host 0.0.0.0 
``` 

con: 

http://127.0.0.1:5001


![assets/mlflow-project-2-docker-part-0](assets/mlflow-project-2-docker-part-0.png)

**Detalles de la corrida**

![assets/mlflow-project-2-docker-part-1](assets/mlflow-project-2-docker-part-1.png)
![assets/mlflow-project-2-docker-part-2](assets/mlflow-project-2-docker-part-2.png)
![assets/mlflow-project-2-docker-part-3](assets/mlflow-project-2-docker-part-3.png)